In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp

In [2]:
pwd

'/Users/shayneufeld/GitHub/mouse_bandit/jupyter_notebooks'

In [3]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
import bandit_preprocessing as bp

In [4]:
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given']

In [6]:
trials = pd.read_csv('/Users/shayneufeld/GitHub/mouse_bandit/data/80_20_trial_data/dumble_harry/dumble_070716_8020_trials.csv',names=columns)

In [7]:
trials.head(10)

,Elapsed Time (s),Since last trial (s),Trial Duration (s),Port Poked,Right Reward Prob,Left Reward Prob,Reward Given
0,36.037,1.801,0.439,2,0.2,0.8,1
1,38.218,1.828,0.353,2,0.2,0.8,1
2,40.551,1.917,0.416,2,0.2,0.8,1
3,43.372,2.160,0.661,2,0.2,0.8,1
4,46.131,2.411,0.348,2,0.2,0.8,1
5,49.824,3.286,0.407,2,0.2,0.8,0
6,52.692,2.405,0.463,2,0.2,0.8,1
7,55.438,2.379,0.367,2,0.2,0.8,1
8,57.997,2.234,0.325,2,0.2,0.8,1
9,60.612,2.133,0.482,2,0.2,0.8,0


In [14]:
mouse_id = 'dumble'
session_id = '070716'

# Important Note:

For now, will only consider 10th trial and onwards for simplicity. 
**May want to revisit this for model that builds its own features for simulating behavior session**

In [10]:
n_trials = trials.shape[0]
n_trials

n_summary = 10;
n_indi = 5;

In [55]:
df = trials.copy()
n_trials = df.shape[0]

n_indi = 10;

num_cols = 4*n_indi + 2 + 1 + 2 #2 streak rows, 4 cols for each past trial, 1 for current trial + 2 decision/switch!
feature_matrix = np.zeros((n_trials-n_indi,num_cols))

In [106]:
for j,i in enumerate(np.arange(n_indi,n_trials)):
    
    #extract the 'n_summary' trials we need to consider. Assume that n_summary > n_indi
    past_trials = df.iloc[i-n_indi:i]
    
    '''
    Mouse ID
    '''
    # will be added after (since its a string)
    
    '''
    Session ID
    '''
    # will be added after (since its a string)
    
    '''
    PORT STREAK
    
    approach: take the derivative of the 'port poked' variabe. Streak is number of [0s + 1] (from the end). 
    the valence of the streak is the sign of first non-zero entry (from the end)
    '''
    streakP_vec = np.flipud(np.diff(past_trials['Port Poked'].values)) #reverse order of array so end is
    #at the front. This makes it easier to find the first non-zero entry
    streakP_len = np.nonzero(streakP_vec)[0]
    
    if len(streakP_len) == 0: #have to deal with case where streak is all 10 previous trials!
        feature_matrix[j,0] = 10
    #otherwise, streak is less then 10 trials and things are simpler.
    else:
        streakP_len = streakP_len[0]
        feature_matrix[j,0] = (streakP_len+1)
    
    
    '''
    REWARD STREAK
    
    approach: take the derivative of the reward boolean. Streak is number of [0s + 1] (from the end). 
    the valence of the streak is the sign of first non-zero entry (from the end)
    '''
    streakR_vec = np.flipud(np.diff(past_trials['Reward Given'].values)) #reverse order of array so end is
    #at the front. This makes it easier to find the first non-zero entry
    streakR_len = np.nonzero(streakR_vec)[0]
    
    if len(streakR_len) == 0: #have to deal with case where streak is all 10 previous trials!
        streakR_len = 10
        if np.sum(past_trials['Reward Given'].values > 0):
            streakR_sign = 1
        else:
            streakR_sign = -1
    #otherwise, streak is less then 10 trials and things are simpler.
    else:
        streakR_len = streakR_len[0]
        streakR_sign = streakR_vec[streakR_len]
    
        feature_matrix[j,1] = (streakR_len+1)*streakR_sign
        
    '''
    INDIVIDUAL TRIALS
    '''
    k = 2
    for icol,itrial in enumerate(np.arange(n_indi,0,-1)):
        
        past_trial = past_trials.iloc[-itrial,:]
        
        #which port
        if past_trial['Port Poked'] == 1:
            feature_matrix[j,k] = 0
        elif past_trial['Port Poked'] == 2:
            feature_matrix[j,k] = 1
        else:
            print('Error port not Left or Right')
        k += 1
        
        #reward given:
        feature_matrix[j,k] = past_trial['Reward Given']
        k += 1
        
        #ITI
        feature_matrix[j,k] = past_trial['Since last trial (s)']
        k += 1
        
        #trial time
        feature_matrix[j,k] = past_trial['Trial Duration (s)']
        k += 1
    
    '''
    CURRENT TRIAL
    '''
    current_trial = df.iloc[i,:]
    feature_matrix[j,k] = current_trial['Since last trial (s)']
    k += 1

    '''
    DECISION
    '''
    if current_trial['Port Poked'] == 1:
        feature_matrix[j,k] = 0
    elif current_trial['Port Poked'] == 2:
        feature_matrix[j,k] = 1
    else:
        print('Error decision port not Left or Right')
    
    k += 1
    '''
    SWITCH
    '''
    feature_matrix[j,k] = np.abs((current_trial['Port Poked'] - df.iloc[i-1]['Port Poked']))

In [102]:
past_trials

,Elapsed Time (s),Since last trial (s),Trial Duration (s),Port Poked,Right Reward Prob,Left Reward Prob,Reward Given
546,1831.3,2.417,0.412,1,0.8,0.2,0
547,1841.1,9.339,0.488,1,0.8,0.2,1
548,1843.4,1.796,0.496,1,0.8,0.2,1
549,1845.3,1.449,0.503,1,0.8,0.2,1
550,1849.3,3.614,0.335,1,0.8,0.2,1
551,1851.4,1.762,0.324,1,0.8,0.2,0
552,1853.4,1.663,0.354,1,0.8,0.2,1
553,1855.7,1.993,0.324,1,0.8,0.2,1
554,1858.1,2.042,0.333,1,0.8,0.2,1
555,1860.3,1.763,0.421,1,0.8,0.2,1


In [104]:
streakP_vec

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [105]:
np.nonzero(streakP_vec)[0]

array([], dtype=int64)

In [98]:
'''
convert into feature dataframe
'''
feature_names = [
                'Port Streak',
                'Reward Streak',
                '10_Port',
                '10_Reward',
                '10_ITI',
                '10_trialDuration',
                '9_Port',
                '9_Reward',
                '9_ITI',
                '9_trialDuration',
                '8_Port',
                '8_Reward',
                '8_ITI',
                '8_trialDuration',
                '7_Port',
                '7_Reward',
                '7_ITI',
                '7_trialDuration',
                '6_Port',
                '6_Reward',
                '6_ITI',
                '6_trialDuration',
                '5_Port',
                '5_Reward',
                '5_ITI',
                '5_trialDuration',
                '4_Port',
                '4_Reward',
                '4_ITI',
                '4_trialDuration',
                '3_Port',
                '3_Reward',
                '3_ITI',
                '3_trialDuration',
                '2_Port',
                '2_Reward',
                '2_ITI',
                '2_trialDuration',
                '1_Port',
                '1_Reward',
                '1_ITI',
                '1_trialDuration',
                '0_ITI',
                'Decision',
                'Switch'
                 ]

## Add mouse ID and session ID

In [113]:
d = {'Mouse ID':mouse_id,'Session ID':session_id}
feature_df = pd.DataFrame(data=d,index=range(feature_matrix.shape[0]))

In [115]:
feature_df = pd.concat([feature_df,pd.DataFrame(data=feature_matrix,index=None,columns=feature_names)],axis=1)
feature_df.head(5)

,Mouse ID,Session ID,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,10_trialDuration,9_Port,9_Reward,...,2_Reward,2_ITI,2_trialDuration,1_Port,1_Reward,1_ITI,1_trialDuration,0_ITI,Decision,Switch
0,dumble,070716,10.0,-1.0,1.0,1.0,1.801,0.439,1.0,1.0,...,1.0,2.234,0.325,1.0,0.0,2.133,0.482,1.411,1.0,0.0
1,dumble,070716,10.0,-2.0,1.0,1.0,1.828,0.353,1.0,1.0,...,0.0,2.133,0.482,1.0,0.0,1.411,0.435,2.169,0.0,1.0
2,dumble,070716,1.0,-3.0,1.0,1.0,1.917,0.416,1.0,1.0,...,0.0,1.411,0.435,0.0,0.0,2.169,0.395,3.838,0.0,0.0
3,dumble,070716,2.0,-4.0,1.0,1.0,2.160,0.661,1.0,1.0,...,0.0,2.169,0.395,0.0,0.0,3.838,0.469,1.846,1.0,1.0
4,dumble,070716,1.0,1.0,1.0,1.0,2.411,0.348,1.0,0.0,...,0.0,3.838,0.469,1.0,1.0,1.846,0.378,1.527,1.0,0.0
